## Update
|version|update|cv|lb|
|:--|:--|--:|--:|
|v3|-|-|0.343|
|v4|increase N_ROWS from 90M to 180M|0.573|0.465|
|v5|increase N_SAMPLES from 1M to 2M<br>change to a larger model (5M -> 10M)|0.622|0.486|
|v8|Add normalization commented by [@hengck23](https://www.kaggle.com/hengck23)|0.629|0.496|

In [1]:
!pip install rdkit

In [2]:
!pip install -U ../data/package/lightning-2.2.1-py3-none-any.whl

Processing /root/program/leash/data/package/lightning-2.2.1-py3-none-any.whl
lightning is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
from pathlib import Path

import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics import AveragePrecision
import lightning as L
from lightning.pytorch.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    TQDMProgressBar,
)
from transformers import AutoConfig, AutoTokenizer, AutoModel, DataCollatorWithPadding
import datasets
from rdkit import Chem

2024-06-05 09:04:18.658782: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 09:04:18.687251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 09:04:18.687273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 09:04:18.688049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-05 09:04:18.694110: I tensorflow/core/platform/cpu_feature_guar

In [54]:
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    
  
    
    
if Config.DEBUG:
    n_rows = 10**4
else:
    n_rows = None
    

In [5]:
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"

TRAIN_DATA_NAME = "local_train_enc.parquet"

In [6]:
DEBUG = True
NORMALIZE = True
N_ROWS = 180_000_000
assert N_ROWS is None or N_ROWS % 3 == 0
if DEBUG:
    N_SAMPLES = 10_000
else:
    N_SAMPLES = 2_000_000
PROTEIN_NAMES = ["BRD4", "HSA", "sEH"]
data_dir = Path(RAW_DIR)
model_name = "DeepChem/ChemBERTa-10M-MTR"
batch_size = 256
trainer_params = {
  "max_epochs": 5,
  "enable_progress_bar": True,
  "accelerator": "auto",
  "precision": "16-mixed",
  "gradient_clip_val": None,
  "accumulate_grad_batches": 1,
  "devices": [0],
}

TRAIN_DATA_NAME = "local_train.parquet"


# Prepare Dataset

In [7]:
df = pl.read_parquet(
    Path(PROCESSED_DIR, TRAIN_DATA_NAME),
    columns=["molecule_smiles", "protein_name", "binds"],
    n_rows=N_ROWS,
)
test_df = pl.read_parquet(
    Path(RAW_DIR, "test.parquet"),
    columns=["molecule_smiles"],
    n_rows=10000 if DEBUG else None,
)

df.head()

molecule_smiles,protein_name,binds
str,str,i64
"""O=C(N[Dy])c1cnn(-c2ccc(F)cc2)c…","""BRD4""",0
"""Cc1csc(Nc2nc(Nc3ncco3)nc(N[C@@…","""sEH""",0
"""COC(=O)c1ccc(Nc2nc(NCc3cccc(C(…","""BRD4""",0
"""Cc1cc(F)ncc1CNc1nc(Nc2nc3ncc(C…","""BRD4""",0
"""C=CCC(CC=C)(Nc1nc(NCC(C)S(C)=O…","""BRD4""",0


In [28]:
dfs = []
for i, protein_name in enumerate(PROTEIN_NAMES):
    sub_df = df[i::3]
    sub_df = sub_df.rename({"binds": protein_name})
    if i == 0:
        dfs.append(sub_df.drop(["id", "protein_name"]))
    else:
        dfs.append(sub_df[[protein_name]])
df = pl.concat(dfs, how="horizontal")
df = df.sample(n=N_SAMPLES)
print(df.head())
print(df[PROTEIN_NAMES].sum())

shape: (5, 4)
┌─────────────────────────────────┬──────┬─────┬─────┐
│ molecule_smiles                 ┆ BRD4 ┆ HSA ┆ sEH │
│ ---                             ┆ ---  ┆ --- ┆ --- │
│ str                             ┆ i64  ┆ i64 ┆ i64 │
╞═════════════════════════════════╪══════╪═════╪═════╡
│ CC1(F)CCN(CCNc2nc(NCC3(CO)CC4C… ┆ 0    ┆ 0   ┆ 0   │
│ COC(=O)c1c[nH]c(C(=O)OC)c1Nc1n… ┆ 0    ┆ 0   ┆ 0   │
│ O=C(N[Dy])c1cnc(Cl)cc1Nc1nc(NC… ┆ 0    ┆ 0   ┆ 0   │
│ O=C(N[Dy])[C@H](Cc1ccsc1)Nc1nc… ┆ 0    ┆ 0   ┆ 0   │
│ O=C(N[Dy])[C@H](Cc1ccc(C(F)(F)… ┆ 0    ┆ 0   ┆ 0   │
└─────────────────────────────────┴──────┴─────┴─────┘
shape: (1, 3)
┌──────┬─────┬─────┐
│ BRD4 ┆ HSA ┆ sEH │
│ ---  ┆ --- ┆ --- │
│ i64  ┆ i64 ┆ i64 │
╞══════╪═════╪═════╡
│ 65   ┆ 51  ┆ 54  │
└──────┴─────┴─────┘


In [29]:
def normalize(x):
    mol = Chem.MolFromSmiles(x)
    smiles = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
    return smiles


if NORMALIZE:
    df = df.with_columns(pl.col("molecule_smiles").map_elements(normalize, return_dtype=pl.Utf8))
    test_df = test_df.with_columns(pl.col("molecule_smiles").map_elements(normalize, return_dtype=pl.Utf8))

In [30]:
train_idx, val_idx = train_test_split(np.arange(len(df)), test_size=0.2)
train_df, val_df = df[train_idx], df[val_idx]
len(train_df), len(val_df)

(8000, 2000)

# Build Dataset

In [31]:
# tokenizer とは、テキストデータを機械学習モデルが理解できる形式に変換するためのツールです。具体的には、文章を単語やサブワードに分割し、それぞれを数値化する役割を果たします。
# 今回はChemBERTa使用
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [32]:
def tokenize(batch, tokenizer):
    output = tokenizer(batch["molecule_smiles"], truncation=True)
    return output


class LMDataset(Dataset):
    def __init__(self, df, tokenizer, stage="train"):
        assert stage in ["train", "val", "test"]
        self.tokenizer = tokenizer
        self.stage = stage
        df = (
            datasets.Dataset
            .from_pandas(df.to_pandas())
            .map(tokenize, batched=True, fn_kwargs={"tokenizer": self.tokenizer})
            .to_pandas()
        )
        self.df = pl.from_pandas(df)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data = self._generate_data(index)
        data["label"] = self._generate_label(index)
        return data        

    def _generate_data(self, index):
        data = {
            "input_ids": np.array(self.df[index, "input_ids"]),
            "attention_mask": np.array(self.df[index, "attention_mask"]),
        }
        return data
    
    def _generate_label(self, index):
        if self.stage == "test":
            return np.array([0, 0, 0])
        else:
            return self.df[index, PROTEIN_NAMES].to_numpy()[0]


LMDataset(train_df[:100], tokenizer)[0]

# input_ids: トークン化されたSMILEのID．もとのSMILES表記がそれぞれのトークンに分割され、それらがIDに変換された結果です
# attention_mask: トークンがパディングされているかどうかを示すマスクです．パディングされている場合は0，それ以外は1です．
# label: 3つのターゲットのバインディング情報です．


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'input_ids': array([12, 16, 15, 20, 15, 15, 17, 23, 15, 21, 25, 15, 17, 23, 16, 15, 26,
        15, 15, 17, 54, 18, 15, 15, 32, 15, 15, 15, 25, 15, 26, 32, 18, 25,
        15, 17, 23, 15, 26, 25, 15, 15, 17, 54, 18, 15, 15, 26, 16, 17, 22,
        19, 18, 23, 18, 25, 21, 18, 25, 17, 31, 15, 21, 15, 15, 15, 15, 15,
        21, 18, 25, 20, 13], dtype=int32),
 'attention_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1], dtype=int8),
 'label': array([0, 0, 0])}

In [33]:
class LBDataModule(L.LightningDataModule):
    def __init__(self, train_df, val_df, test_df, tokenizer):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer

    def _generate_dataset(self, stage):
        if stage == "train":
            df = self.train_df
        elif stage == "val":
            df = self.val_df
        elif stage == "test":
            df = self.test_df
        else:
            raise NotImplementedError
        dataset = LMDataset(df, self.tokenizer, stage=stage)
        return dataset

    def _generate_dataloader(self, stage):
        dataset = self._generate_dataset(stage)
        if stage == "train":
            shuffle=True
            drop_last=True
        else:
            shuffle=False
            drop_last=False
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            drop_last=drop_last,
            pin_memory=True,
            collate_fn=DataCollatorWithPadding(self.tokenizer),
        )

    def train_dataloader(self):
        return self._generate_dataloader("train")

    def val_dataloader(self):
        return self._generate_dataloader("val")

    def test_dataloader(self):
        return self._generate_dataloader("test")
    
    def local_test_dataloader(self, local_test_df):
        dataset = LMDataset(local_test_df, self.tokenizer, stage="test")
        return DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            drop_last=False,
            pin_memory=True,
            collate_fn=DataCollatorWithPadding(self.tokenizer),
        )
    
    
datamodule = LBDataModule(train_df, val_df, test_df, tokenizer)

# Build Model

In [34]:
class LMModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name, num_labels=3)
        # hugging faceのtransformersライブラリを使用して、事前学習済みモデルをロード
        self.lm = AutoModel.from_pretrained(model_name, add_pooling_layer=False)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.config.hidden_size, self.config.num_labels)
        self.loss_fn = nn.BCEWithLogitsLoss(reduction="mean")

    def forward(self, batch):
        last_hidden_state = self.lm(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
        ).last_hidden_state
        logits = self.classifier(
            self.dropout(last_hidden_state[:, 0])
        )
        return {
            "logits": logits,
        }

    def calculate_loss(self, batch):
        output = self.forward(batch)
        loss = self.loss_fn(output["logits"], batch["labels"].float())
        output["loss"] = loss
        return output

    
LMModel(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


LMModel(
  (lm): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(600, 384, padding_idx=1)
      (position_embeddings): Embedding(515, 384, padding_idx=1)
      (token_type_embeddings): Embedding(1, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.144, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-2): 3 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.109, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,)

In [35]:
class LBModelModule(L.LightningModule):
    def __init__(self, model_name):
        super().__init__()
        self.model = LMModel(model_name)
        self.map = AveragePrecision(task="binary")

    def forward(self, batch):
        return self.model(batch)

    def calculate_loss(self, batch, batch_idx):
        return self.model.calculate_loss(batch)

    def training_step(self, batch, batch_idx):
        ret = self.calculate_loss(batch, batch_idx)
        self.log("train_loss", ret["loss"], on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return ret["loss"]

    def validation_step(self, batch, batch_idx):
        ret = self.calculate_loss(batch, batch_idx)
        self.log("val_loss", ret["loss"], on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.map.update(F.sigmoid(ret["logits"]), batch["labels"].long())

    def on_validation_epoch_end(self):
        val_map = self.map.compute()
        self.log("val_map", val_map, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        self.map.reset()

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        logits = self.forward(batch)["logits"]
        probs = F.sigmoid(logits)
        return probs

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        return {
            "optimizer": optimizer,
        }

    
modelmodule = LBModelModule(model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Training

In [36]:
checkpoint_callback = ModelCheckpoint(
    filename=f"model-{{val_map:.4f}}",
    save_weights_only=True,
    monitor="val_map",
    mode="max",
    dirpath=MODEL_DIR,
    save_top_k=1,
    verbose=1,
)
early_stop_callback = EarlyStopping(monitor="val_map", mode="max", patience=3)
progress_bar_callback = TQDMProgressBar(refresh_rate=1)
callbacks = [
    checkpoint_callback,
    early_stop_callback,
    progress_bar_callback,
]

In [37]:
trainer = L.Trainer(callbacks=callbacks, **trainer_params)
trainer.fit(modelmodule, datamodule)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:215: UserWarning: 
NVIDIA H100 PCIe with CUDA capability sm_90 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA H100 PCIe GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(
INFO: You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

...

INFO: Epoch 2, global step 93: 'val_map' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 3, global step 124: 'val_map' reached 0.00956 (best 0.00956), saving model to '/root/program/leash/models/model-val_map=0.0096.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 4, global step 155: 'val_map' was not in top 1
INFO: `Trainer.fit` stopped: `max_epochs=5` reached.


# Inference

In [38]:
test_df = pl.read_parquet(
    Path(data_dir, "test.parquet"),
    columns=["molecule_smiles"],
    n_rows=10000 if DEBUG else None,
)

In [39]:
working_dir = Path(MODEL_DIR)
model_paths = working_dir.glob("*.ckpt")
test_dataloader = datamodule.test_dataloader()
for model_path in model_paths:
    print(model_path)
    modelmodule = LBModelModule.load_from_checkpoint(
        checkpoint_path=model_path,
        model_name=model_name,
    )
    predictions = trainer.predict(modelmodule, test_dataloader)
    predictions = torch.cat(predictions).numpy()
    pred_dfs = []
    for i, protein_name in enumerate(PROTEIN_NAMES):
        pred_dfs.append(
            test_df.with_columns(
                pl.lit(protein_name).alias("protein_name"),
                pl.lit(predictions[:, i]).alias("binds"),
            )
        )
    pred_df = pl.concat(pred_dfs)
    submit_df = (
        pl.read_parquet(Path(data_dir, "test.parquet"), columns=["id", "molecule_smiles", "protein_name"])
        .join(pred_df, on=["molecule_smiles", "protein_name"], how="left")
        .select(["id", "binds"])
        .sort("id")
    )
    submit_df.write_csv(Path(working_dir, f"submission_{model_path.stem}.csv"))

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

../models/model-val_map=0.0946.ckpt


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

/tmp/ipykernel_349201/1186736651.py:22: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  pl.read_parquet(Path(data_dir, "test.parquet"), columns=["id", "molecule_smiles", "protein_name"])


...

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

../models/model-val_map=0.0096.ckpt


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [48]:
local_test_df = pl.read_parquet(
    Path(PROCESSED_DIR, "local_test.parquet"),
    columns=["molecule_smiles", "protein_name", "binds"],
    n_rows=10000 if DEBUG else None,
)

def one_hot_binds(df):
    df = df.with_columns([
    ((df["protein_name"] == "BRD4").cast(pl.Int32) * df["binds"]).alias("BRD4"),
    ((df["protein_name"] == "HSA").cast(pl.Int32) * df["binds"]).alias("HSA"),
    ((df["protein_name"] == "sEH").cast(pl.Int32) * df["binds"]).alias("sEH")
    ])
    df = df.drop(["binds", "protein_name"])
    return df
local_test_df = one_hot_binds(local_test_df)


target = local_test_df[PROTEIN_NAMES]
local_test_df = local_test_df.drop(PROTEIN_NAMES)

In [49]:
datamodule.local_test_df = local_test_df
local_test_dataloader = datamodule.local_test_dataloader(local_test_df)

for model_path in model_paths:
    print(model_path)
    modelmodule = LBModelModule.load_from_checkpoint(
        checkpoint_path=model_path,
        model_name=model_name,
    )
    local_predictions = trainer.predict(modelmodule, local_test_dataloader)
    local_predictions = torch.cat(local_predictions).numpy()
    local_pred_dfs = []
    for i, protein_name in enumerate(PROTEIN_NAMES):
        local_pred_dfs.append(
            local_test_df.with_columns(
                pl.lit(protein_name).alias("protein_name"),
                pl.lit(local_predictions[:, i]).alias("binds"),
            )
        )
    local_pred_df = pl.concat(local_pred_dfs)
    local_submit_df = (
        local_test_df
        .select(["id", "molecule_smiles", "protein_name"])
        .join(local_pred_df, on=["molecule_smiles", "protein_name"], how="left")
        .select(["id", "binds"])
        .sort("id")
    )
    local_submit_df.write_csv(Path(working_dir, f"local_submission_{model_path.stem}.csv"))


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [52]:
from sklearn.metrics import average_precision_score

# ローカルテストデータに対する推論の実行と結果データフレームの作成
def infer_and_calculate_ap(local_test_df, model_paths, datamodule, trainer, model_name, PROTEIN_NAMES):
    local_test_dataloader = datamodule.local_test_dataloader(local_test_df)
    results = []

    for model_path in model_paths:
        print(model_path)
        modelmodule = LBModelModule.load_from_checkpoint(
            checkpoint_path=model_path,
            model_name=model_name,
        )
        local_predictions = trainer.predict(modelmodule, local_test_dataloader)
        local_predictions = torch.cat(local_predictions).numpy()
        local_pred_dfs = []
        for i, protein_name in enumerate(PROTEIN_NAMES):
            local_pred_dfs.append(
                local_test_df.with_columns(
                    pl.lit(protein_name).alias("protein_name"),
                    pl.lit(local_predictions[:, i]).alias("predicted_binds"),
                )
            )
        local_pred_df = pl.concat(local_pred_dfs)
        local_submit_df = (
            local_test_df
            .select(["id", "molecule_smiles", "protein_name", "binds"])
            .join(local_pred_df, on=["molecule_smiles", "protein_name"], how="left")
            .select(["id", "molecule_smiles", "protein_name", "binds", "predicted_binds"])
            .sort("id")
        )
        results.append(local_submit_df)

    return results

# 平均適合率スコアの計算
def calculate_average_precision(local_submit_df, PROTEIN_NAMES):
    average_precision_scores = {}
    for protein_name in PROTEIN_NAMES:
        y_true = local_submit_df.filter(pl.col("protein_name") == protein_name)["binds"].to_numpy()
        y_scores = local_submit_df.filter(pl.col("protein_name") == protein_name)["predicted_binds"].to_numpy()
        average_precision_scores[protein_name] = average_precision_score(y_true, y_scores)

    return average_precision_scores

# 推論の実行とAPスコアの計算
results = infer_and_calculate_ap(local_test_df, model_paths, datamodule, trainer, model_name, PROTEIN_NAMES)

for i, result in enumerate(results):
    print(f"Results for model {i+1}:")
    print(result)

    # APスコアの計算
    ap_scores = calculate_average_precision(result, PROTEIN_NAMES)
    for protein_name, ap_score in ap_scores.items():
        print(f"Average Precision Score for {protein_name}: {ap_score:.4f}")

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Ensemble

In [ ]:
sub_files = list(working_dir.glob("submission_*.csv"))
sub_files

[PosixPath('../models/submission_model-val_map=0.0946.csv')]

In [ ]:
sub_dfs = []
for sub_file in sub_files:
    sub_dfs.append(pl.read_csv(sub_file))
submit_df = (
    pl.concat(sub_dfs)
    .group_by("id")
    .agg(pl.col("binds").mean())
    .sort("id")
)

In [ ]:
submit_df.write_csv(Path(OUTPUT_DIR, "submission.csv"))

## Future Directions
- Finding the optimal CV strategy
- Increase data
- Utilize buildingblock1 ~ buildingblock3
- Large-scale models
- Tune hyper-parameters
- Ensemble
- etc...